In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# First : exploring the data

We'll need to have a look at what the data is, how it is distributed for the different features, and start to get an intuition about what methods might work better for analysis and prediction later.

## Load the training data into feature matrix, class labels, and event ids:

In [ ]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [ ]:
# remove samples with error values
idx_c = np.all(tX!=-999, axis=1)
y_c = y[idx_c]
tX_c = tX[idx_c]
# regularize
mean = np.mean(tX_c, axis=0)
std = np.std(tX_c, axis=0)
tX_c = (tX_c-mean)/std

In [ ]:
print("Overall: s: ",np.sum(y==1),", b: ",np.sum(y==-1)," ,total:",len(y))
print("NoErrors: s: ",np.sum(y_c==1),", b: ",np.sum(y_c==-1)," ,total:",len(y_c))
for n in range(tX_c.shape[1]):
    plt.figure(figsize=(20,4))
    plt.subplot(131)
    plt.hist([tX_c[y_c==1,n],tX_c[y_c==-1,n]], 20, density=True, histtype='bar', stacked=True)
    plt.legend(['s','b'])
    plt.title('Feature '+str(n))
    plt.subplot(132)
    plt.title('s histogram feature '+str(n))
    plt.hist(tX_c[y_c==1,n], 20, density=True, histtype='bar', stacked=True)
    plt.subplot(133)
    plt.title('b histogram feature '+str(n))
    plt.hist(tX_c[y_c==-1,n], 20, density=True, histtype='bar', stacked=True)    
    plt.show()

In [ ]:
# remove features with error values
idx_gf = np.arange(tX.shape[1])[np.all(tX!=-999, axis=0)]
y_gf = y
tX_gf = tX[:,idx_gf]
# regularize
mean = np.mean(tX_gf, axis=0)
std = np.std(tX_gf, axis=0)
tX_gf = (tX_gf-mean)/std

In [ ]:
print("Overall: s: ",np.sum(y==1),", b: ",np.sum(y==-1)," ,total:",len(y))
for n in range(tX_gf.shape[1]):
    plt.figure(figsize=(20,4))
    plt.subplot(131)
    plt.hist([tX_gf[y_gf==1,n],tX_gf[y_gf==-1,n]], 20, density=True, histtype='bar', stacked=True)
    plt.legend(['s','b'])
    plt.title('Feature '+str(idx_gf[n]))
    plt.subplot(132)
    plt.title('s histogram feature '+str(idx_gf[n]))
    plt.hist(tX_gf[y_gf==1,n], 20, density=True, histtype='bar', stacked=True)
    plt.subplot(133)
    plt.title('b histogram feature '+str(idx_gf[n]))
    plt.hist(tX_gf[y_gf==-1,n], 20, density=True, histtype='bar', stacked=True)    
    plt.show()

# Actual predictions start from here

After having looked at the data we will now do some actual predictions using different models andd parameters.

## Load the test data

In [ ]:
from proj1_helpers import *
DATA_TEST_PATH = '../data/test.csv.zip' 
#test_y, test_X, ids = load_csv_data(DATA_TRAIN_PATH)

## Generate predictions using only features with no errrors throughought

This enables us to use some of the methods from the course directly, without having to adjust some of the functionnality to account for the fact that a lot of errors are in the dataset. First let us see which features from the test dataset are error free.

In [ ]:
# Import test data

import pandas as pd

test_x_df = pd.read_csv(DATA_TEST_PATH, index_col='Id').drop('Prediction', axis='columns')
test_x_df

train_x_df = pd.read_csv(DATA_TRAIN_PATH, index_col='Id').drop('Prediction', axis='columns')
train_y_df = pd.read_csv(DATA_TRAIN_PATH, index_col='Id', usecols=['Id', 'Prediction'])

In [ ]:
# Remove columns with -999 errors
error_cols_te = np.full(test_x_df.shape[1], False)
for i in range(test_x_df.shape[1]):
#     print(test_x_df.iloc[:,i].values)
#     print(-999.0 in test_x_df.iloc[:,i].values)
    if -999.0 in test_x_df.iloc[:,i].values:
        error_cols_te[i] = True
print(f"There are {error_cols_te.tolist().count(True)} test error columns are which are : \n{error_cols_te}\n")

error_cols_tr = np.full(train_x_df.shape[1], False)
for i in range(train_x_df.shape[1]):
    if -999.0 in train_x_df.iloc[:,i].values:
        error_cols_tr[i] = True
print(f"There are {error_cols_tr.tolist().count(True)} train error columns are which are : \n{error_cols_tr}")

no_error_cols_tr_te = (~(error_cols_tr | error_cols_te)).tolist()
print(f"They have {no_error_cols_tr_te.count(True)} following shared non error columns :\n{no_error_cols_tr_te}")

With this figured out we can now extract the valid columns from test and train data, do some training and testing on data, then generate answers for the test data and submit to aicrowd !

# Save prediction ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)